In [ ]:
!pip install ultralytics


In [ ]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
def read_csv ( csv_path ):
    np_path_XYs = np . genfromtxt ( csv_path , delimiter =',')
    path_XYs = []
    for i in np . unique ( np_path_XYs [: , 0]):
        npXYs = np_path_XYs [ np_path_XYs [: , 0] == i ][: , 1:]
        XYs = []
        for j in np . unique ( npXYs [: , 0]):
            XY = npXYs [ npXYs [: , 0] == j ][: , 1:]
            XYs.append(XY)
        path_XYs . append ( XYs )
    return path_XYs


def cubic_bezier(t, P0, P1, P2, P3):
    return (1 - t)**3 * P0 + 3 * (1 - t)**2 * t * P1 + 3 * (1 - t) * t**2 * P2 + t**3 * P3

def bezier_from_polyline(points):
    bezier_curves = []
    for i in range(len(points) - 1):
        P0 = points[i]
        P3 = points[i + 1]

        if i == 0:
            P1 = P0 + 0.4 * (P3 - P0)
        else:
            P1 = P0 + 0.4 * (P3 - P0)

        if i == len(points) - 2:
            P2 = P3 - 0.4 * (P3 - P0)
        else:
            P2 = P3 - 0.4 * (P3 - P0)

        bezier_curves.append((P0, P1, P2, P3))

    return bezier_curves


curves = bezier_from_polyline(firstimg[0][0])

# Plotting
t = np.linspace(0, 1, 100)
plt.figure()
for curve in curves:
    P0, P1, P2, P3 = curve
    bezier_curve = np.array([cubic_bezier(i, P0, P1, P2, P3) for i in t])
    plt.plot(bezier_curve[:, 0], bezier_curve[:, 1], 'b-')
    plt.scatter([P0[0], P1[0], P2[0], P3[0]], [P0[1], P1[1], P2[1], P3[1]], color='red')

plt.title('Cubic Bezier Approximation of Polyline')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.show()

fig=plt.figure()
for i in firstimg:
    curves = bezier_from_polyline(i[0])
    for curve in curves:
        P0, P1, P2, P3 = curve
        bezier_curve = np.array([cubic_bezier(i, P0, P1, P2, P3) for i in t])
        plt.plot(bezier_curve[:, 0], bezier_curve[:, 1], 'b-')
        plt.scatter([P0[0], P1[0], P2[0], P3[0]], [P0[1], P1[1], P2[1], P3[1]], color='red')

plt.title('Cubic Bezier Approximation of Polyline')
plt.xlabel('x')
plt.ylabel('y')
plt.show()
fig.savefig(r'C:\Users\ACER\Downloads\curve1.png',bbox_inches='tight',pad_inches=0)

# Load the trained model for saved images and detecting the path through instance segmentation
model = YOLO('C:\Users\ACER\Downloads\CurvetopiaForAdobeGenai\instanc.pt') # path for the png images
import cv2
# Load the image
img = cv2.imread("../content/drive/MyDrive/adobe_projectforgenai/curve1.png")

# Make predictions on the image
results = model(img)
print(results)

# Iterate over the results
for result in results:
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
    scores = result.boxes.conf.cpu().numpy()  # Confidence scores
    classes = result.boxes.cls.cpu().numpy().astype(int)

    # Iterate over the detections
    for box, score, cls in zip(boxes, scores, classes):
        x1, y1, x2, y2 = [int(c) for c in box]
        class_name = model.names[cls]
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, f"{class_name} {score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

# Display the image with predictions
cv2.imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
from scipy import optimize

def fit_circle_least_squares(x, y):
    def calc_R(xc, yc):
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f_2(c):
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = np.mean(x), np.mean(y)
    center, _ = optimize.leastsq(f_2, center_estimate)

    xc, yc = center
    Ri = calc_R(*center)
    R = Ri.mean()

    return xc, yc, R

def adaptive_circle_detection(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (9, 9), 2)

    height, width = gray.shape
    min_dim = min(height, width)

    min_radius = int(min_dim * 0.1)
    max_radius = int(min_dim * 0.5)

    best_circle = None
    best_score = 0

    for dp in [1, 2]:
        for param1 in range(50, 301, 50):
            for param2 in range(10, 101, 10):\
                circles = cv.HoughCircles(
                    blurred,
                    cv.HOUGH_GRADIENT,
                    dp=dp,
                    minDist=min_dim//4,
                    param1=param1,
                    param2=param2,
                    minRadius=min_radius,
                    maxRadius=max_radius
                )

                if circles is not None:
                    circles = np.uint16(np.around(circles))
                    for circle in circles[0, :]:
                        x, y, r = circle
                        mask = np.zeros(gray.shape, dtype=np.uint8)
                        cv.circle(mask, (x, y), r, 255, 1)
                        edge_response = cv.mean(cv.Canny(gray, 100, 200), mask=mask)[0]
                        score = edge_response * r  # Favoring larger circles with strong edges

                        if score > best_score:
                            best_score = score
                            best_circle = circle

    return best_circle
img = cv.imread('C:\Users\ACER\Downloads\CurvetopiaForAdobeGenai\curve1.png')

best_circle = adaptive_circle_detection(img)

# Loading the trained YOLO model
model = YOLO('C:\Users\ACER\Downloads\CurvetopiaForAdobeGenai\instanc.pt')

results = model(img)

all_points = []

if best_circle is not None:
    x, y, r = best_circle
    theta = np.linspace(0, 2*np.pi, 100)
    circle_x = x + r*np.cos(theta)
    circle_y = y + r*np.sin(theta)
    all_points.extend(list(zip(circle_x, circle_y)))

for result in results:
    masks = result.masks.data.cpu().numpy()
    for mask in masks:
        y, x = np.where(mask)
        all_points.extend(list(zip(x, y)))
print(results)
all_points = np.array(all_points)

if len(all_points) > 0:
    xc, yc, R = fit_circle_least_squares(all_points[:, 0], all_points[:, 1])

    cv.circle(img, (int(xc), int(yc)), int(R), (0, 255, 0), 2)

    cv2_imshow(img)
    print(f"Best-fit circle: Center = ({xc:.2f}, {yc:.2f}), Radius = {R:.2f}")

    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    cv.circle(mask, (int(xc), int(yc)), int(R), 255, -1)

    result = cv.bitwise_and(img, img, mask=mask)

    cv2_imshow(result)
else:
    print("No points detected for circle fitting.")

cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
from scipy import optimize
import csv


# After fitting the circle and displaying the results converting it to polyline csv file

if len(all_points) > 0:
    xc, yc, R = fit_circle_least_squares(all_points[:, 0], all_points[:, 1])

    theta = np.linspace(0, 2*np.pi, 100)
    x = xc + R * np.cos(theta)
    y = yc + R * np.sin(theta)

    x = np.round(x).astype(int)
    y = np.round(y).astype(int)

    polyline_points = np.column_stack((x, y))

    csv_filename = 'polyline_points.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['x', 'y'])
        csv_writer.writerows(polyline_points)

    print(f"Polyline points saved to {csv_filename}")

    for point in polyline_points:
        cv.circle(img, tuple(point), 1, (0, 0, 255), -1)

    cv2_imshow(img)
else:
    print("No points detected for circle fitting.")

cv.waitKey(0)
cv.destroyAllWindows()